##### <p> Samuel Wolfe <br> September 30, 2023 <br> MSBA 207 <br> Chapter 10a </p>

In [20]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math
from sklearn.preprocessing import StandardScaler

## DMBA Chapter 10 - Problem 1

#### Loading the data and cutting down to the requisite predictors

In [21]:
#keepers = ['Selected','Default','New','RealEstate','DisbursementGross','Portion','Recession']
drops = ['Obs','TotCap/Assets']

DATA = Path('E:/Aliit/School/MSBA/206/MSBA-206/dmba')

banks_df = pd.read_csv(DATA / 'Banks.csv')
banks_df = banks_df.drop(columns=drops)
banks_df

,Financial Condition,TotExp/Assets,TotLns&Lses/Assets
0,1,0.12,0.65
1,1,0.11,0.62
2,1,0.09,1.02
3,1,0.10,0.67
4,1,0.11,0.69
5,1,0.13,0.74
6,1,0.10,0.79
7,1,0.13,0.63
8,1,0.16,0.72
9,1,0.16,0.57


In [22]:
y = banks_df['Financial Condition']
X = banks_df.drop(columns=['Financial Condition'])
#X2 = sm.add_constant(X, prepend=True)
# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

In [23]:
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear', tol=1e-8)
logit_reg.fit(X,y)
print('intercept', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())
print('AIC', AIC_score(y, logit_reg.predict(X), df = len(X.columns) +1))

intercept -14.72100809095756
       TotExp/Assets  TotLns&Lses/Assets
coeff      89.833916             8.37132
AIC 18.705839468305996


#### <p> 10.1.a where b_0 is expected result when variables are 0, b_1 is coef of 'TotExp/Assets', b_2 is coef of 'TotLns&Lses/Assets', iv_1 is 'TotExp/Assets', and iv_2 is 'TotLns&Lses/Assets'.<br>(i) logit = b_0 + b_1 * iv_1 + b_2 * iv_2<br>logit = -14.7210 + (89.8339 * TotExp/Assets) + (8.3713 * TotLns&Lses) <br>(ii) Odds(Financial Condition = 1 | TotExp/Assets = iv_1 & TotLns&Lses/Assets = iv_2) = 1/(e^(b_0 + b_1 * iv_1 + b_2 * iv_2))<br> Odds = e^[-14.7210 + (89.8339 * TotExp/Assets) + (8.3713 * TotLns&Lses)]<br>(iii) Prob = odds/(1+odds)<br>prob = 1/(1 + e^[-14.7210 + (89.8339 * TotExp/Assets) + (8.3713 * TotLns&Lses)])</p>  

In [24]:
testData = pd.DataFrame({'TotExp/Assets': [0.11],
                         'TotLns&Lses/Assets': [0.6]})
testData

,TotExp/Assets,TotLns&Lses/Assets
0,0.11,0.6


In [25]:
TotExpAssets = 0.11
TotLnsLses = 0.6

#### <p>10.1.b See below.</p>

In [26]:
logit = round(-14.7210 + (89.8339 * TotExpAssets) + (8.3713 * TotLnsLses), 6)
Odds = round(math.exp(logit), 6)
prob = round(Odds/(1 + Odds), 6)

In [27]:
logit

0.183509

In [28]:
Odds

1.201426

In [29]:
prob

0.545749

#### 10.1.c If the cutoff is p=.5 then the odds will be 1. Which means if the odds are greater than 1 then the status will be weak. then with logit will be 0 meaning if the logit output is greater than 0 status will be weak.

#### 10.1.d Looking at the coef for the 'TotLns&Lses/Assets', -3.5993, it is telling us that for every 1 unit gain in this variable value the odds of being 'Financial Condition' = 0 goes up by 4321.3364.

In [30]:
math.exp(8.37132)

4321.336463737168

In [31]:
math.exp(89.833916)

1.0336511842979622e+39

#### 10.1.e Given that misclassification cost is HIGHER when a bank with poor financial condition is misclassified as if it is in good financial condition, it would be best to decrease the threshold as we are classifying for poor financial condition. This might lower the accuracy of the model, but in this case it is better to miss opportunities than misclassify a poor financial condition bank.

## 10.4

In [32]:
#keepers = ['Selected','Default','New','RealEstate','DisbursementGross','Portion','Recession']
drops = ['Obs','TotCap/Assets']

DATA = Path('E:/Aliit/School/MSBA/206/MSBA-206/dmba')

ebay_df = pd.read_csv(DATA / 'eBayAuctions.csv')
#banks_df = banks_df.drop(columns=drops)
pd.set_option('display.width', 200)
print(ebay_df)
print(ebay_df.dtypes)

              Category currency  sellerRating  Duration endDay  ClosePrice  OpenPrice  Competitive?
0     Music/Movie/Game       US          3249         5    Mon        0.01       0.01             0
1     Music/Movie/Game       US          3249         5    Mon        0.01       0.01             0
2     Music/Movie/Game       US          3249         5    Mon        0.01       0.01             0
3     Music/Movie/Game       US          3249         5    Mon        0.01       0.01             0
4     Music/Movie/Game       US          3249         5    Mon        0.01       0.01             0
...                ...      ...           ...       ...    ...         ...        ...           ...
1967        Automotive       US          2992         5    Sun      359.95     359.95             0
1968        Automotive       US            21         5    Sat      610.00     300.00             1
1969        Automotive       US          1400         5    Mon      549.00     549.00             0


In [33]:
ebay_df['Category'] = ebay_df['Category'].astype('category')
ebay_df['currency'] = ebay_df['currency'].astype('category')
ebay_df['Duration'] = ebay_df['Duration'].astype('category')
ebay_df['endDay'] = ebay_df['endDay'].astype('category')
print(ebay_df.dtypes)

Category        category
currency        category
sellerRating       int64
Duration        category
endDay          category
ClosePrice       float64
OpenPrice        float64
Competitive?       int64
dtype: object


In [34]:
table1 = pd.pivot_table(ebay_df, index=['Category'], values='Competitive?', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive?')])
table1

C:\Users\Sam\AppData\Local\Temp\ipykernel_44808\119725634.py:1: FutureWarning: The provided callable <function mean at 0x0000023B7F2553A0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  table1 = pd.pivot_table(ebay_df, index=['Category'], values='Competitive?', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive?')])


,mean
,Competitive?
Category,
Health/Beauty,0.171875
EverythingElse,0.235294
Coins/Stamps,0.297297
Pottery/Glass,0.350000
Automotive,0.353933
Jewelry,0.365854
Books,0.500000
Clothing/Accessories,0.504202


In [35]:
table2 = pd.pivot_table(ebay_df, index=['currency'], values='Competitive?', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive?')])
table2

C:\Users\Sam\AppData\Local\Temp\ipykernel_44808\3604635627.py:1: FutureWarning: The provided callable <function mean at 0x0000023B7F2553A0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  table2 = pd.pivot_table(ebay_df, index=['currency'], values='Competitive?', aggfunc=[np.mean]).sort_values(by=[('mean','Competitive?')])


,mean
,Competitive?
currency,
US,0.519350
EUR,0.551595
GBP,0.687075


#### I could not get the pivot table to display anything legible.


In [36]:
ebay_df.replace(['Sun','Wed','Fri'],'Sun, Wed, Fri', inplace=True)
ebay_df.replace(['Business/Industrial','Computer','Home/Garden'],'Business/Industrial, Computer, and Home/Garden', inplace=True)
ebay_df.replace(['Antique/Art/Craft', 'Collectibles'],'Antique/Art/Craft and Collectibles', inplace=True)
ebay_df.replace(['Automotive','Pottery/Glass'],'Automotive and Pottery/Glass', inplace=True)
ebay_df.replace(['Books','Clothing/Accessories'],'Books and Clothing/Accessories', inplace=True)
ebay_df

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
...,...,...,...,...,...,...,...,...
1967,Automotive and Pottery/Glass,US,2992,5,"Sun, Wed, Fri",359.95,359.95,0
1968,Automotive and Pottery/Glass,US,21,5,Sat,610.00,300.00,1
1969,Automotive and Pottery/Glass,US,1400,5,Mon,549.00,549.00,0
1970,Automotive and Pottery/Glass,US,57,7,"Sun, Wed, Fri",820.00,650.00,1


In [37]:
ebay_df_dum = pd.get_dummies(ebay_df,drop_first=False)

#### 10.4.b Found below

In [38]:
y = ebay_df_dum['Competitive?']
X = ebay_df_dum.drop(columns=['Competitive?'])

# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
# Initiate scaler
sc = StandardScaler()
# Standardize the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(train_X),index=train_X.index, columns=train_X.columns)
# Standardized the testing dataset
X_valid_transformed = pd.DataFrame(sc.transform(valid_X),index=valid_X.index, columns=valid_X.columns)

In [39]:
X_train_transformed_X2 = sm.add_constant(X_train_transformed, prepend=True)
X_valid_transformed_X2 = sm.add_constant(X_valid_transformed, prepend=True)
logit_full = sm.GLM(train_y, X_train_transformed_X2, family=sm.families.Binomial())
logit_result = logit_full.fit()


ebayFull = logit_result.predict(X_valid_transformed_X2)
ebayFull_nominal = [ 0 if x < 0.5 else 1 for x in ebayFull]
classificationSummary(valid_y, ebayFull_nominal)

Confusion Matrix (Accuracy 0.7529)

       Prediction
Actual   0   1
     0 290  63
     1 132 304


C:\Users\Sam\AppData\Roaming\Python\Python312\site-packages\statsmodels\genmod\families\links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


#### 10.4.c Found below

In [40]:
#X_train_transformed_X2 = sm.add_constant(X_train_transformed.drop(columns='ClosePrice'), prepend=True)
logit_part = sm.GLM(train_y, X_train_transformed.drop(columns='ClosePrice'), family=sm.families.Binomial())
logit_result_part = logit_part.fit()

ebayPart = logit_result_part.predict(X_valid_transformed.drop(columns='ClosePrice'))
ebayPart_nominal = [ 0 if x < 0.5 else 1 for x in ebayPart]
classificationSummary(valid_y, ebayPart_nominal)

Confusion Matrix (Accuracy 0.6286)

       Prediction
Actual   0   1
     0 236 117
     1 176 260


#### 10.4.c If we compare the accuracy of the two models, we see that our accuracy has gone down. This can be seen as a good thing though as the question explains, we cannot know the closing price. We can however create a model to predict what the closing price may be given the items category and other data, thus allowing us to predict if the auction will be competitive.

In [41]:
logit_result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           Competitive?   No. Observations:                 1183
Model:                            GLM   Df Residuals:                     1157
Model Family:                Binomial   Df Model:                           25
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -600.60
Date:                Mon, 23 Oct 2023   Deviance:                       1201.2
Time:                        17:54:13   Pearson chi2:                 1.48e+08
No. Iterations:                   100   Pseudo R-squ. (CS):             0.3070
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
const                                                    3.046e+13    1.6e+05    1.9e+08      0.000    3.05e+13    3.05e+13
sellerRating                                               -0.2599      0.098     -2.639      0.008      -0.453      -0.067
ClosePrice                                                  7.3635      0.833      8.837      0.000       5.730       8.997
OpenPrice                                                  -4.2135      0.456     -9.239      0.000      -5.107      -3.320
Category_Antique/Art/Craft and Collectibles             -1.431e+13   7.54e+04   -1.9e+08      0.000   -1.43e+13   -1.43e+13
Category_Automotive and Pottery/Glass                   -1.064e+13   5.61e+04   -1.9e+08      0.000   -1.06e+13   -1.06e+13
Category_Books and Clothing/Accessories                 -9.914e+12   5.22e+04   -1.9e+08      0.000   -9.91e+12   -9.91e+12
Category_Business/Industrial, Computer, and Home/Garden -9.519e+12   5.02e+04   -1.9e+08      0.000   -9.52e+12   -9.52e+12
Category_Coins/Stamps                                   -4.813e+12   2.54e+04   -1.9e+08      0.000   -4.81e+12   -4.81e+12
Category_Electronics                                    -5.481e+12   2.89e+04   -1.9e+08      0.000   -5.48e+12   -5.48e+12
Category_EverythingElse                                 -3.346e+12   1.76e+04   -1.9e+08      0.000   -3.35e+12   -3.35e+12
Category_Health/Beauty                                  -6.597e+12   3.48e+04   -1.9e+08      0.000    -6.6e+12    -6.6e+12
Category_Jewelry                                        -6.451e+12    3.4e+04   -1.9e+08      0.000   -6.45e+12   -6.45e+12
Category_Music/Movie/Game                                 -1.4e+13   7.37e+04   -1.9e+08      0.000    -1.4e+13    -1.4e+13
Category_Photography                                     3.662e+14   1.93e+06    1.9e+08      0.000    3.66e+14    3.66e+14
Category_SportingGoods                                  -8.753e+12   4.61e+04   -1.9e+08      0.000   -8.75e+12   -8.75e+12
Category_Toys/Hobbies                                   -1.083e+13    5.7e+04   -1.9e+08      0.000   -1.08e+13   -1.08e+13
currency_EUR                                                1.2014      0.070     17.246      0.000       1.065       1.338
currency_GBP                                                1.1202      0.127      8.807      0.000       0.871       1.369
currency_US                                                 1.5961      0.070     22.799      0.000       1.459       1.733
Duration_1                                                  0.1154      0.086      1.345      0.179      -0.053       0.284
Duration_3                                                  0.3645      0.071      5.142      0.000       0

#### 10.4.d Looking at the coef for Closing Price, 7.3635, we see that for every unit increase of closing price, it increases the odds of the auction being competitive by 7.3635. Given its listed P score is 0.000, its statistically significant for the relationship between closing price and competitiveness.